# 1 - Exploration et nettoyage

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", None)


On commence par créer un dataframe à partir du fichier CSV fourni.

In [ ]:
df = pd.read_csv('flickr_data.csv')

In [ ]:
df.info()

On enlève les espaces en début de nom de colonne.

In [ ]:
df.columns = df.columns.str.strip()

## Colonnes de dates

Les colonnes de dates sont converties en numérique.

In [ ]:
date_cols = [
    "date_taken_minute", "date_taken_hour", "date_taken_day",
    "date_taken_month", "date_taken_year",
    "date_upload_minute", "date_upload_hour", "date_upload_day",
    "date_upload_month", "date_upload_year",
]

for c in date_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

On s'intéresse à la cohérence de ces colonnes.

In [ ]:
date_rules = {
    "date_taken_minute": (0, 59),
    "date_taken_hour": (0, 23),
    "date_taken_day": (1, 31),
    "date_taken_month": (1, 12),
    "date_taken_year": (1900, 2026),

    "date_upload_minute": (0, 59),
    "date_upload_hour": (0, 23),
    "date_upload_day": (1, 31),
    "date_upload_month": (1, 12),
    "date_upload_year": (1900, 2026),
}

outlier_mask = pd.Series(False, index=df.index)

for col, (lo, hi) in date_rules.items():
    outlier_mask |= ~df[col].between(lo, hi)

df_outliers = df[outlier_mask]
df_outliers.head(5)

Plusieurs colonnes contiennent des valeurs aberrantes (trop petites ou trop grandes) qui ne correspondent pas à des dates valides.
On décide de créer de nouvelles colonnes de type datetime : les lignes avec des valeurs aberrantes seront converties en NaT (Not a Time).

Si on a besoin de faire des analyses sur la date, on les fera en utilisant ces nouvelles colonnes nettoyées.
On ne supprime pas les lignes avec des valeurs aberrantes, car elles peuvent contenir des informations utiles dans d'autres colonnes.

In [ ]:
df["taken_dt"] = pd.to_datetime(
    dict(
        year=df["date_taken_year"],
        month=df["date_taken_month"],
        day=df["date_taken_day"],
        hour=df["date_taken_hour"],
        minute=df["date_taken_minute"],
    ),
    errors="coerce"
)

df["upload_dt"] = pd.to_datetime(
    dict(
        year=df["date_upload_year"],
        month=df["date_upload_month"],
        day=df["date_upload_day"],
        hour=df["date_upload_hour"],
        minute=df["date_upload_minute"],
    ),
    errors="coerce"
)

df[["taken_dt", "upload_dt"]].info()

## Colonnes Unnamed

On décide de supprimer les colonnes inutiles "Unnamed: 16", "Unnamed: 17" et "Unnamed: 18" car elles ne contiennent aucune information pertinente pour notre analyse, et sont globalement vides.

In [ ]:
df.drop(columns=['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'], inplace=True)
df.info()

## Lignes dupliquées

Maintenant que l'on a supprimé les colonnes inutiles, on peut vérifier la présence de lignes dupliquées dans le DataFrame.

In [ ]:
n_row_dupes = df.duplicated().sum()
n_row_dupes

Il y a 252 139 lignes dupliquées dans le DataFrame. On les supprime.

In [ ]:
df = df.drop_duplicates() # Garde la première occurence
df.info()

On tombe à 168 101 lignes uniques.

## Colonnes latitude, longitude

On vérifie la cohérence des colonnes "latitude" et "longitude".

In [ ]:
df[["lat", "long"]].describe()

On veut visualiser le carré encadrant toutes les coordonnées GPS présentes dans le dataset, pour voir si toutes les données sont bien à Lyon.

In [ ]:
# On veut visualiser le carré encadrant toutes les coordonnées GPS présentes dans le dataset sur une carte
min_lat, max_lat = df["lat"].min(), df["lat"].max()
min_long, max_long = df["long"].min(), df["long"].max()

import folium
m = folium.Map(location=[(min_lat + max_lat) / 2, (min_long + max_long) / 2], zoom_start=2)
m.fit_bounds([[min_lat, min_long], [max_lat, max_long]])
m

Toutes les données sont à Lyon et ses environs.

## Colonnes title, tags

On s'intéresse maintenant aux colonnes title et tags.

In [ ]:
df[["title", "tags"]].tail(10)

On commence par un nettoyage basique de ces colonnes :
- On les convertit en type string
- On enlève les espaces superflus au début et fin
- On remplace les espaces multiples par un seul dans la colonne "title" et "tags"

In [ ]:
for c in ["title", "tags", "user"]:
    df[c] = df[c].astype("string")
    df[c] = df[c].fillna("").str.strip()
df["title"] = df["title"].str.replace(r"\s+", " ", regex=True)
df["tags"] = df["tags"].str.replace(r"\s+", " ", regex=True)

On crée une nouvelle colonne "title_tags" qui concatène les colonnes "title" et "tags", séparées par des virgules.

Les tags qui contiennent des espaces sont séparés en plusieurs tags.

Par exemple :
"titre,tag1,tag2,tag3,composé"

In [ ]:
df["title_tags"] = (
    df["title"].str.replace(" ", ",") + "," + 
    df["tags"].str.replace(" ", ",")
)
df["title_tags"].tail(10)

Le text mining sera basé sur cette colonne. Un nettoyage plus avancé sera fait plus tard.

## Colonne URL

On ajoute la colonne "url" qui sera utile pour accéder aux images et examiner le contenu du dataset.

In [ ]:
df["url"] = (
    "https://www.flickr.com/photos/"
    + df["user"].astype(str)
    + "/"
    + df["id"].astype(str)
)
df["url"].tail(10)

# Export du DataFrame nettoyé

On exporte le dataframe nettoyé dans un fichier .parquet pour une utilisation dans les étapes suivantes.

In [ ]:
# Export en parquet dans /output
df.to_parquet("output/data_cleaned.parquet", index=False)